>># **Ciencia de Datos en Python**
### Proyecto Final
###### *Marco Tulio Escobar Muñoz*

In [ ]:
import numpy as np
import panda as pd
import matplotlib as mt
import seaborn as sb
import scikit-learn as sl

class my_regresion(self, x, y):
    def __init__():
        self.beta0 = []
        self.beta1 = []
        self.x = []
        self.y = []
        self.archivo = np.load("proyecto_training_data.npy")
        